In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import time 

In [2]:
def readucr(filename):#读取数据
    data = np.loadtxt(filename, dtype=str, delimiter=',') 
    x = np.array(data[:, 0:575], dtype=np.float32)
    y = np.array(data[:, 1:576], dtype=np.float32)
    x = x.reshape(x.shape[0], -1, 1)
    y = y.reshape(y.shape[0], -1, 1)
    print(f'{filename}: {x.shape}, {y.shape}')
    return x, y

In [3]:
class Autoencoder_GRU: # GRU神经网络
    """
        model初始化
    """
    def __init__(self, output_directory, input_shape, output_shape, verbose= True): 
        self.output_directory = output_directory
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.verbose = verbose
        
        self.model = self.build_model() # 调用 build_model
        if(verbose == True):
            self.model.summary() # 打印 model 结构
        if not os.path.exists(self.output_directory):
            os.mkdir(self.output_directory)
        init_model_file = os.path.join(self.output_directory, 'model_init.hdf5')
        self.model.save_weights(init_model_file) # 保存初始权重

    """
       创建model
    """         
    def build_model(self): 
        # model 网络结构
        input_layer = tf.keras.Input(self.input_shape)
        layer_1 = tf.keras.layers.GRU(20, return_sequences=True)(input_layer) 
        layer_2 = tf.keras.layers.BatchNormalization()(layer_1)
        layer_3 = tf.keras.layers.Activation(activation='tanh', name='gru_output')(layer_2)
        output_layer = tf.keras.layers.Dense(self.output_shape)(layer_3)

        model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
        # 编译model
        model.compile(loss='mse', optimizer='adam',metrics=['mae']) 
        
        # 设置 callbacks 回调
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', # 用于动态调整learning rate
                                                         factor=0.5, 
                                                         patience=50, 
                                                         min_lr=0.0001)

        file_path =  os.path.join(self.output_directory, 'best1_model.hdf5')

        model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=file_path, 
                                                              monitor='loss', 
                                                              save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model 

    # 训练model
    def fit(self, x_train, y_train, x_val, y_val, epochs): 
        """
            验证集: (x_val, y_val) 用于监控loss，防止overfitting
        """  
        batch_size = 16
        mini_batch_size = int(min(x_train.shape[0]/10, batch_size))

        start_time = time.time() 

        hist = self.model.fit(x_train, y_train, 
                              validation_data=(x_val, y_val),
                              batch_size=mini_batch_size, 
                              epochs=epochs,
                              verbose=self.verbose,  
                              callbacks=self.callbacks)
        
        duration = time.time() - start_time

        tf.keras.backend.clear_session() # 清除当前tf计算图


In [4]:
X_train,Y_train = readucr('Car_TRAIN.txt')
X_test,Y_test = readucr('Car_TEST.txt')

Car_TRAIN.txt: (60, 575, 1), (60, 575, 1)
Car_TEST.txt: (60, 575, 1), (60, 575, 1)


In [5]:
input_shape = X_train.shape[1:]

In [6]:
model = Autoencoder_GRU('test', input_shape, 1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 575, 1)]          0         
_________________________________________________________________
gru (GRU)                    (None, 575, 20)           1380      
_________________________________________________________________
batch_normalization (BatchNo (None, 575, 20)           80        
_________________________________________________________________
gru_output (Activation)      (None, 575, 20)           0         
_________________________________________________________________
dense (Dense)                (None, 575, 1)            21        
Total params: 1,481
Trainable params: 1,441
Non-trainable params: 40
_________________________________________________________________


In [7]:
model.fit(X_train, Y_train, X_test, Y_test, 5)

Train on 60 samples, validate on 60 samples
Epoch 1/5
60/60 [==============================] - 8s 134ms/sample - loss: 1.2880 - mae: 1.0031 - val_loss: 1.0847 - val_mae: 0.9172
Epoch 2/5
60/60 [==============================] - 4s 61ms/sample - loss: 0.3738 - mae: 0.5041 - val_loss: 0.9469 - val_mae: 0.8567
Epoch 3/5
60/60 [==============================] - 4s 59ms/sample - loss: 0.0353 - mae: 0.1443 - val_loss: 0.8511 - val_mae: 0.8118
Epoch 4/5
60/60 [==============================] - 4s 58ms/sample - loss: 0.0384 - mae: 0.1630 - val_loss: 0.8204 - val_mae: 0.7970
Epoch 5/5
60/60 [==============================] - 4s 59ms/sample - loss: 0.0394 - mae: 0.1674 - val_loss: 0.8248 - val_mae: 0.7996


In [8]:
initial_model = tf.keras.models.load_model('test/best1_model.hdf5')

In [9]:
gru_output_model = tf.keras.Model(
    inputs = initial_model.input,
    outputs = initial_model.get_layer('gru_output').output
)

In [10]:
gru_output_model.predict(X_test)

array([[[-0.1226807 , -0.00656664, -0.03303735, ..., -0.12119589,
         -0.13136786, -0.19281803],
        [-0.16349672,  0.00175988, -0.05068986, ..., -0.169845  ,
         -0.17041363, -0.28307983],
        [-0.17361991,  0.01070093, -0.06663083, ..., -0.18729421,
         -0.18337937, -0.32570148],
        ...,
        [-0.1596286 ,  0.05563255, -0.10797611, ..., -0.19153723,
         -0.20483914, -0.37481895],
        [-0.15891242,  0.05577553, -0.10764334, ..., -0.19084422,
         -0.20509815, -0.3738011 ],
        [-0.16210368,  0.05631377, -0.10788301, ..., -0.19409893,
         -0.20843814, -0.37699157]],

       [[-0.14440955, -0.00451682, -0.03570873, ..., -0.13772254,
         -0.14907391, -0.21614803],
        [-0.1921086 ,  0.00555703, -0.05628344, ..., -0.19470684,
         -0.192021  , -0.31671846],
        [-0.20485632,  0.01601491, -0.07491984, ..., -0.21679302,
         -0.20614912, -0.3647179 ],
        ...,
        [-0.198812  ,  0.06451052, -0.12091839, ..., -